이제는 알바천국을 조진다.

In [8]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
import re


result = pd.DataFrame()  # 결과저장용 DataFrame
pagenum = 1

while(len(result) <= 100):  # 결과가 100개 넘어야 함.
    url = 'http://www.alba.co.kr/job/area/MainLocal.asp?page=argu&pagesize=50&viewtype=L&sidocd=031&gugun=%BD%C3%C8%EF%BD%C3,&dong=%C0%FC%C3%BC&d_area=&d_areacd=&strAreaMulti=031%7C%7C%BD%C3%C8%EF%BD%C3%7C%7C%C0%FC%C3%BC%2C&hidJobKind=&hidJobKindMulti=&WorkTime=&searchterm=&AcceptMethod=&ElecContract=&HireTypeCD=&CareerCD=&CareercdUnRelated=&LastSchoolCD=&LastSchoolcdUnRelated=&GenderCD=&GenderUnRelated=&AgeLimit=0&AgeUnRelated=&PayCD=&PayStart=&WelfareCD=&Special=&WorkWeekCD=&WeekDays=&hidSortCnt=50&hidSortOrder=&hidSortDate=&WorkPeriodCD=&hidSort=&hidSortFilter=Y&hidListView=LIST&WsSrchKeywordWord=&hidWsearchInOut=&hidSchContainText='
    url = url.replace('argu', str(pagenum))  # 페이지 1에대한 경우 이후 이는 변수로 처리할 것.
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')

    # crawltags = '#NormalInfo > table > tbody > tr:nth-child(1)'
    number = 1  # 99까지가 한페이지 사이클임
    while number < 99:
        title = soup.select_one(
            '#NormalInfo > table > tbody > tr:nth-child(%d)' % number)

        #근무지역에 대한 컬럼
        locations = str(re.sub(
            '<.+?>', '', str(title.select('td.local.first')), 0).strip())
        locations = locations.replace('[' , '')
        locations = locations.replace(']' , '')
        company = re.sub(
            '<.+?>', '', str(title.select('td.title > a > span.company')), 0).strip()
        company = company.replace('[' , '')
        company = company.replace(']' , '')
        
        #근무시간에 대한 컬럼
        time = re.sub('<.+?>', '', str(title.select('td.data')), 0).strip()
        time = time.replace('[' , '')
        time = time.replace(']' , '')

        #급여에 대한 컬럼
        paymethod = re.sub('<.+?>', '', str(title.select('td.pay')), 0).strip()
        paymethod = paymethod.replace('[' , '')
        paymethod = paymethod.replace(']' , '')
        #올린시간에 대한 컬럼
        regdate = re.sub(
            '<.+?>', '', str(title.select('td.regDate.last')), 0).strip()

        regdate = regdate.replace('[' , '')
        regdate = regdate.replace(']' , '')
        #알바설명에 대한 컬럼
        info= re.sub(
            '<.+?>', '', str(title.select('td.title > a > span.title')), 0).strip()

        info = info.replace('[' , '')
        info = info.replace(']' , '')

        df = pd.DataFrame([[locations, company, time, paymethod, regdate, info]], columns=[
                          '지역', '근무회사', '근무시간', '급여', '올린시간','알바설명'])
        number += 2  # 다음꺼 접근해야지
        # 알바가 아닌 직원을 구하는 글에대한 필터링 작업 이거에 대해서 삭제작업 추가해야함.
        if paymethod.find("월급") == 1 or paymethod.find("연봉") == 1:
            print("cut!!")
            # idx = result[result['급여형태']>1800000].index
            # result = result.drop(idx)
            continue  # 월급준다는 거면 일단 제끼기 알바가 아닐 확률이 높기 때문임.
        else:
            result = result.append(df)

    result = result.drop_duplicates(['근무회사'], keep='first')  # 중복글에 대한 처리
    
    
    result['순서'] = range(len(result))
    result = result.set_index('순서')
    print(len(result))
    
    idx = result[result['급여'].str.contains('월급')].index ## df[조건식]
    result = result.drop(idx)
    idx = result[result['급여'].str.contains('연봉')].index ## df[조건식]
    result = result.drop(idx)

    #없앴으니까 다시 인덱싱 해주기
    result['순서'] = range(len(result))
    result = result.set_index('순서')

    pagenum = pagenum + 1
    print(result)

    #후처리로 drop_duplicate 사용해서 중복되는 것들 제거할 것

#to_csv' 인덱스는 제거할 것
result.to_csv('albaheaven.csv', encoding='CP949', index=False)


46
46
          지역            근무회사                근무시간            급여   올린시간  \
순서                                                                      
1     시흥시 전체         COUPANG                시간협의  연봉35,000,000  06/30   
2     시흥시 전체              쿠팡                시간협의      시급17,000  06/30   
12   시흥시 정왕동      60계치킨시흥정왕점  18:00~23:00 (협의가능)       시급8,720  06/29   
14   시흥시 정왕동          선유탁송대리                시간협의     일급200,000  06/30   
16   시흥시 정왕동        CU 시화라성점         14:00~19:00       시급8,720  06/28   
17   시흥시 정왕동    교촌치킨 시흥 정왕1호         17:30~23:00       시급8,720  06/30   
19   시흥시 목감동       GS25목감샘터점         00:00~07:00       시급8,720  06/30   
20    시흥시 전체     CU 은계주공7단지점         00:00~07:00       시급8,720  06/29   
22   시흥시 정왕동      다이소 시흥정왕본점  08:00~14:00 (로테이션)       시급8,720  06/28   
23   시흥시 정왕동     다이소 시흥정왕2호점  08:00~14:00 (로테이션)       시급8,720  06/28   
24   시흥시 월곶동       다이소 시흥월곶점  09:00~15:00 (로테이션)       시급8,720  06/28   
26   시흥시 월곶동  세븐일레븐 시흥월곶중앙로점  14:00~23:00 (협의

In [9]:
result[result['급여'].str.contains('월급')]

,지역,근무회사,근무시간,급여,올린시간,알바설명
순서,,,,,,


In [10]:
result

,지역,근무회사,근무시간,급여,올린시간,알바설명
순서,,,,,,
0,시흥시 전체,COUPANG,시간협의,"연봉35,000,000",06/30,"쿠팡친구(전 쿠팡맨) 채용(월400가능/주5일/신입,여성가능)"
1,시흥시 전체,쿠팡,시간협의,"시급17,000",06/30,삼성 비스포크&amp;첫배송 13만원 보너스 지급
2,시흥시 정왕동,60계치킨시흥정왕점,18:00~23:00 (협의가능),"시급8,720",06/29,60계치킨정왕점 주말알바모집
3,시흥시 정왕동,선유탁송대리,시간협의,"일급200,000",06/30,"탁송기사,대리기사/일급보장/ 남녀거주지 우선배정/초보가능/알바가능"
4,시흥시 정왕동,CU 시화라성점,14:00~19:00,"시급8,720",06/28,성실한 주말 근무자 모집
...,...,...,...,...,...,...
102,시흥시 정왕동,CU 배곧로얄2호점,22:00~08:00,"시급8,720",8시간전,CU배곧로얄2호점 평일야간 STAFF 모집합니다.
103,시흥시 정왕동,CU 배곧로얄1호점,22:00~08:00,"시급8,720",8시간전,CU배곧로얄1호점 평일야간 STAFF를 모집합니다.
104,시흥시 전체,CU 배곧호반점,19:00~07:00,"시급8,720",06/30,평일 야간 근무자 급구
